In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import networkx as nx

from src.utils.utils_networks import global_efficiency

In [2]:
files_path = '../data/processed/'
year=2015

network_path = f'{files_path}{year}/migration_network.graphml'
G = nx.readwrite.graphml.read_graphml(network_path)